In [ ]:
# !pip install pymysql
# !pip show sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.6 MB/s eta 0:00:00
Name: SQLAlchemy
Version: 2.0.40
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: greenlet, typing-extensions
Required-by: ipython-sql, langchain


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
### 데이터베이스 접속정보
user = ""
password = ""
dbip = ""
dbport = ""
database = "project2025"
dbConPath = f"mysql+pymysql://{user}:{password}@{dbip}:{dbport}/{database}"
dbConPath
myengine = create_engine(dbConPath)

In [ ]:
# prompt: 구글드라이브 연동

# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# %cd /content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/과제

/content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/과제


In [ ]:
# csv 정보 mysql에 입력

accountInfo = pd.read_csv("/home/ubuntu/pywork/dataset/account_info.csv")

try:
    tableName = "account_info"
    accountInfo.to_sql(name = tableName, con=myengine, index=False, if_exists="replace")
    accDf = pd.read_sql_query( sql = f"SELECT * FROM {tableName}", con = myengine )
except Exception as e:
    print(e)

customerInfo = pd.read_csv("/home/ubuntu/pywork/dataset/customer_info.csv")
try:
    tableName = "customer_info"
    customerInfo.to_sql(name = tableName, con=myengine, index=False, if_exists="replace")
    custDf = pd.read_sql_query( sql = f"SELECT * FROM {tableName}", con = myengine )
except Exception as e:
    print(e)

loanInfo = pd.read_csv("/home/ubuntu/pywork/dataset/loan_info.csv")
try:
    tableName = "loan_info"
    loanInfo.to_sql(name = tableName, con=myengine, index=False, if_exists="replace")
    #loanDf = pd.read_sql_query( sql = f"SELECT * FROM {tableName}", con = myengine )
    loanDf = pd.read_sql_query( sql = "SELECT customer_id, SUM(loan_amount) AS total_loan_amount FROM loan_info GROUP BY customer_id", con = myengine )

except Exception as e:
    print(e)

transactionInfo = pd.read_csv("/home/ubuntu/pywork/dataset/transaction_info.csv")
try:

    tableName = "transaction_info"
    transactionInfo.to_sql(name = tableName, con=myengine, index=False, if_exists="replace")
    transDf = pd.read_sql_query( sql = "SELECT account_id, SUM(amount) AS total_transaction_amount, Max(transaction_date) AS recent_transaction_date FROM transaction_info GROUP BY account_id", con = myengine )


except Exception as e:
    print(e)

(pymysql.err.OperationalError) (3730, "Cannot drop table 'customer_info' referenced by a foreign key constraint 'customer_finance_summary_ibfk_1' on table 'customer_finance_summary'.")
[SQL: 
DROP TABLE customer_info]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
# # 데이터 불러와서 DF에 담기
# accDf =  pd.read_sql_query( sql = "select * from account_info", con = myengine )
# custDf = pd.read_sql_query( sql = "select * from customer_info", con = myengine )

# #customerId별 loan 합계
# loanDf = pd.read_sql_query( sql = "SELECT customer_id, SUM(loan_amount) AS total_loan_amount FROM loan_info GROUP BY customer_id", con = myengine )

# #customerId별 transaction 합계
# transDf = pd.read_sql_query( sql = "SELECT account_id, SUM(amount) AS total_transaction_amount, Max(transaction_date) AS recent_transaction_date FROM transaction_info GROUP BY account_id", con = myengine )


In [ ]:
# accDf.head(2)

In [ ]:
#custDf.head(2)

In [ ]:
#loanDf.head(2)

In [ ]:
#transDf.head(2)

In [ ]:

joinKey = ["customer_id"]
joinData = pd.merge( left=accDf,
                     right=custDf,
                     on=joinKey,
                     how="inner")
#joinData.head(2)


In [ ]:
joinKey = ["customer_id"]
joinData2 = pd.merge( left=joinData,
                     right=loanDf,
                     on=joinKey,
                     how="inner")
#joinData2.head(2)


In [ ]:
joinKey2 = ["account_id"]
mergedData = pd.merge( left=joinData2,
                     right=transDf,
                     on=joinKey2,
                     how="inner")
#mergedData.head(2)

In [ ]:
#데이터 보정 transaction_date 월표기(010,011,012)
mergedData['recent_transaction_date'] = mergedData['recent_transaction_date'].astype(str)
mergedData['recent_transaction_date'] = mergedData['recent_transaction_date'].str.replace('010', '10')
mergedData['recent_transaction_date'] = mergedData['recent_transaction_date'].str.replace('011', '11')
mergedData['recent_transaction_date'] = mergedData['recent_transaction_date'].str.replace('012', '12')
#mergedData.head(3)

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

#dt_korea = datetime.now(ZoneInfo("Asia/Seoul"))
dt_korea = datetime.now()
time_tag = dt_korea.strftime('%Y-%m-%d %H:%M:%S')
time_tag

'2025-05-07 14:23:28'

In [ ]:
dataColumns = ["customer_id","name","balance","recent_transaction_date", "total_transaction_amount", "total_loan_amount"]
finalDf = mergedData[dataColumns].copy()
finalDf["time_tag"] = time_tag
finalDf = finalDf[["time_tag","customer_id","name","balance","recent_transaction_date", "total_transaction_amount", "total_loan_amount" ]]
finalDf = finalDf.rename(columns={ "balance": "total_balance" })
#finalDf

In [ ]:
try:
    tableName = "customer_finance_summary"
    finalDf.to_sql(name = tableName, con=myengine, index=False, if_exists="append")
    print("DB저장 완료")
except Exception as e:
    print(e)

DB저장 완료


In [ ]:
# 검증 : customer_id 수 확인

cntsql = """ SELECT count(customer_id) as cnt
FROM customer_info"""

cntTargetsql = """SELECT count(distinct(customer_id)) as target_cnt
FROM customer_finance_summary"""


try:
    cntData = pd.read_sql_query( sql = cntsql, con = myengine )
    cntTargetData = pd.read_sql_query( sql = cntTargetsql, con = myengine )
    cntOriginValue = cntData['cnt'].iloc[0]
    cntTargetValue = cntTargetData['target_cnt'].iloc[0]

    if cntOriginValue == cntTargetValue:
        print("고객별 데이터 수 매칭 확인")
    else:
         print("고객별 데이터 수 매칭 에러")
except Exception as e:
    print(e)


고객별 데이터 수 매칭 확인


In [ ]:
# 검증 : 중복 확인

dupsql = """SELECT
    time_tag,
    customer_id,
    COUNT(*) AS cnt
FROM customer_finance_summary
GROUP BY time_tag, customer_id
HAVING COUNT(*) > 1"""

try:
    checkData = pd.read_sql_query( sql = dupsql, con = myengine )
except Exception as e:
    print(e)

if len(checkData) > 0 :
    print("중복")
else:
    print("중복없음")


중복없음


In [ ]:
# 검증 : Null 확인

# nullsql = """SELECT *
# FROM customer_finance_summary
# WHERE time_tag IS NULL
# AND   customer_id IS NULL
# AND   name IS NULL
# AND   total_balance IS NULL
# AND   recent_transaction_date IS NULL
# AND   total_transaction_amount IS NULL"""

# try:
#     nullData = pd.read_sql_query( sql = nullsql, con = myengine )
#     print(len(nullData))
# except Exception as e:
#     print(e)

# if len(nullData) > 0 :
#     print("Not Null 문제 발생")
# else:
#     print("Not Null 없음")



In [ ]:
# 검증 : 총 거래액 합계 확인

sumsql = """select sum(balance) as total
from account_info"""


sumTargetsql = """select sum(total_balance) as target_total
from customer_finance_summary
where date(time_tag) = curdate()"""

try:
    sumData = pd.read_sql_query( sql = sumsql, con = myengine )
    sumTargetData = pd.read_sql_query( sql = sumTargetsql, con = myengine )
    sumOriginValue = sumData['total'].iloc[0]
    sumTargetValue = sumTargetData['target_total'].iloc[0]

    if  sumOriginValue == sumTargetValue:
        print("총 거래액 매칭 확인")
    else:
        print("총 거래액 매칭 에러")
except Exception as e:
    print(e)


총 거래액 매칭 에러


### SQL 조인 후 불러오는 방식

In [ ]:
# SQL
# sel = """SELECT A.customer_id,
#         A.name,
#         SUM(B.balance) AS total_balance,
#         MAX(C.transaction_date) AS recent_transaction_date,
#         SUM(C.amount) AS total_transaction_amount,
#         SUM(D.loan_amount) AS total_loan_amount
#         FROM customer_info A
#         LEFT JOIN account_info B ON A.customer_id = B.customer_id
#         LEFT JOIN transaction_info C ON B.account_id = C.account_id
#         LEFT JOIN loan_info D ON A.customer_id = D.customer_id
#         GROUP BY A.customer_id, A.name"""


In [ ]:
# AnalysisDf = pd.read_sql_query( sql = sel, con = myengine )


In [ ]:
#데이터 오류 정정
# AnalysisDf['recent_transaction_date'] = AnalysisDf['recent_transaction_date'].str.replace('010', '10')
# AnalysisDf['recent_transaction_date'] = AnalysisDf['recent_transaction_date'].str.replace('011', '11')
# AnalysisDf['recent_transaction_date'] = AnalysisDf['recent_transaction_date'].str.replace('012', '12')

In [ ]:
# from datetime import datetime
# from zoneinfo import ZoneInfo

# #dt_korea = datetime.now(ZoneInfo("Asia/Seoul"))
# dt_korea = datetime.now()
# time_tag = dt_korea.strftime('%Y-%m-%d %H:%M:%S')
# time_tag

In [ ]:
# try:
#     tableName = "customer_finance_summary"
#     AnalysisDf.to_sql(name = tableName, con=myengine, index=False, if_exists="append")
#     print("DB저장 완료")
# except Exception as e:
#     print(e)